In [41]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [42]:
my_data = pd.read_csv("Breast_Cancer.csv", delimiter=",")
my_data[0:5]

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [43]:
my_data['Status'].unique()

array(['Alive', 'Dead'], dtype=object)

In [44]:
my_data['6th Stage'].unique()

array(['IIA', 'IIIA', 'IIIC', 'IIB', 'IIIB'], dtype=object)

In [45]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

In [46]:
df=pd.DataFrame(my_data)

In [47]:
for col in df.columns:
    if not pd.api.types.is_integer_dtype(df[col]):  # Check if column is not integer type
        print(f"Value counts for {col}:")
        print(df[col].value_counts())
        print()  # Print an empty line for separation

Value counts for Race:
White    3413
Other     320
Black     291
Name: Race, dtype: int64

Value counts for Marital Status:
Married      2643
Single        615
Divorced      486
Widowed       235
Separated      45
Name: Marital Status, dtype: int64

Value counts for T Stage :
T2    1786
T1    1603
T3     533
T4     102
Name: T Stage , dtype: int64

Value counts for N Stage:
N1    2732
N2     820
N3     472
Name: N Stage, dtype: int64

Value counts for 6th Stage:
IIA     1305
IIB     1130
IIIA    1050
IIIC     472
IIIB      67
Name: 6th Stage, dtype: int64

Value counts for differentiate:
Moderately differentiated    2351
Poorly differentiated        1111
Well differentiated           543
Undifferentiated               19
Name: differentiate, dtype: int64

Value counts for Grade:
2                        2351
3                        1111
1                         543
 anaplastic; Grade IV      19
Name: Grade, dtype: int64

Value counts for A Stage:
Regional    3932
Distant       92
Nam

In [48]:
numerical_columns = df.select_dtypes(include=['int', 'float']).columns

# Describe numerical columns
numerical_stats = df[numerical_columns].describe()

print("Descriptive statistics for numerical columns:")
print(numerical_stats)

Descriptive statistics for numerical columns:
               Age   Tumor Size  Regional Node Examined  \
count  4024.000000  4024.000000             4024.000000   
mean     53.972167    30.473658               14.357107   
std       8.963134    21.119696                8.099675   
min      30.000000     1.000000                1.000000   
25%      47.000000    16.000000                9.000000   
50%      54.000000    25.000000               14.000000   
75%      61.000000    38.000000               19.000000   
max      69.000000   140.000000               61.000000   

       Reginol Node Positive  Survival Months  
count            4024.000000      4024.000000  
mean                4.158052        71.297962  
std                 5.109331        22.921430  
min                 1.000000         1.000000  
25%                 1.000000        56.000000  
50%                 2.000000        73.000000  
75%                 5.000000        90.000000  
max                46.000000       107

In [50]:
columns_to_drop=['T Stage ','N Stage','6th Stage','Race','Grade','A Stage','Estrogen Status','Regional Node Examined','Reginol Node Positive']
df.drop(columns=columns_to_drop, inplace=True)  # Use inplace=True to modify the original DataFrame


In [51]:
df

,Age,Marital Status,differentiate,Tumor Size,Progesterone Status,Survival Months,Status
0,68,Married,Poorly differentiated,4,Positive,60,Alive
1,50,Married,Moderately differentiated,35,Positive,62,Alive
2,58,Divorced,Moderately differentiated,63,Positive,75,Alive
3,58,Married,Poorly differentiated,18,Positive,84,Alive
4,47,Married,Poorly differentiated,41,Positive,50,Alive
...,...,...,...,...,...,...,...
4019,62,Married,Moderately differentiated,9,Positive,49,Alive
4020,56,Divorced,Moderately differentiated,46,Positive,69,Alive
4021,68,Married,Moderately differentiated,22,Negative,69,Alive
4022,58,Divorced,Moderately differentiated,44,Positive,72,Alive
